In [1]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import math 

PROJECT = 'qwiklabs-gcp-aebfb78fe0f1b1d1' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'sarah-bucket' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
os.environ['TFVERSION'] = '1.8'  # Tensorflow version

SEQ_LEN = 50

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Define the project, Bucket and Region and set them.

In [2]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

In [3]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [4]:
! pwd
! ls

/content/datalab/energy_forcasing/Sarah
ARIMA.csv		 Build_model_task.ipynb  Energy_ts	  test.json
BIG_q.ipynb		 CNN_W2P_model		 hyperparam.yaml  trained
build_local_model.ipynb  data			 sarah_data.csv


In [5]:
## move the csv file to the bucket ~ you rly dont need to do this
## !gsutil cp -p /content/datalab/energy_forcasing/Sarah/sarah_data.csv gs://sarah-bucket/sarah_data.csv

In [6]:
CSV_COLUMNS = ['prediction_date', 'wind_speed_100m', 
  'wind_direction_100m', 
  'temperature', 
  'air_density',
  'pressure', 'precipitation', 'wind_gust', 'radiation', 
  'wind_speed', 'wind_direction', 'price']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df5 = pd.read_csv('/content/datalab/energy_forcasing/Sarah/sarah_data.csv', header = None, names = CSV_COLUMNS)
df5.describe()

wind_speed_100m  wind_direction_100m  temperature  air_density  \
count      6646.000000          6646.000000  6646.000000  6646.000000   
mean          4.799901           189.582975    14.920355     1.132338   
std           1.964154            58.126639     9.626527     0.039333   
min           1.372222            47.883333    -6.044444     1.036111   
25%           3.534722           142.598611     7.540278     1.106111   
50%           4.477778           191.908333    14.213889     1.131667   
75%           5.544444           232.043056    21.062500     1.160556   
max          15.044444           329.544444    41.044444     1.229444   

          pressure  precipitation    wind_gust    radiation   wind_speed  \
count  6646.000000    6646.000000  6646.000000  6646.000000  6646.000000   
mean    935.286695       0.117364     4.872509   225.250182     3.220986   
std       6.105083       0.245561     2.428587   266.299815     1.319704   
min     913.800000       0.000000     1.194444     0.000000     0.855556   
25%     932.034722       0.000000     3.284722     0.000000     2.294444   
50%     934.761111       0.000000     4.327778    95.000000     2.961111   
75%     937.805556       0.100000     5.627778   416.422222     3.850000   
max     961.238889       1.700000    16.705556   954.288889     9.738889   

       wind_direction        price  
count     6646.000000  6646.000000  
mean       189.082629    50.153506  
std         56.712796    12.448392  
min         50.538889     4.000000  
25%        143.750000    43.000000  
50%        191.208333    50.890000  
75%        230.865278    59.950000  
max        327.711111    85.050000

In [7]:
df5.shape[0]

6646

In [8]:
#for ii in range(10):
#   N = 7
#  SEQ = df5.price[(0+N*ii):(N+N*ii)]
np.random.seed(seed=None)
# np.random.uniform(0,6646-N)

In [9]:
def to_csv(filename, start, N, J):
  with open(filename, 'w') as ofp:
    for ii in xrange(start, N):
      # nn = int(round(np.random.uniform(low = 0, high = 6466-J),0))
      seq = (df5.price[(ii):(ii+J)]/60) ## trade nn for ii to make an in seq dataset instead of random
      line = ",".join(map(str, seq))
      ofp.write(line + '\n')
      #print(len(seq))
      if len(seq) != J:
        break

import os
try:
  os.makedirs('data/CNN_W2P_model/')
except OSError:
  pass
to_csv('data/CNN_W2P_model/train-1.csv', start = 0, N = 5000, J = 50)  # 1000 sequences
to_csv('data/CNN_W2P_model/valid-1.csv', start = 5001, N = 5101, J = 50)

In [10]:
df2 = pd.read_csv('data/CNN_W2P_model/train-1.csv', header = None)
df2.head()
#df2.describe()
# df2.count()
# df2.shape
## print(df2.iloc[0], df2.iloc[1], df2.iloc[2], )

0         1         2         3         4         5         6   \
0  0.948333  0.942500  0.912333  0.500000  0.781000  0.654167  0.808333   
1  0.942500  0.912333  0.500000  0.781000  0.654167  0.808333  0.724833   
2  0.912333  0.500000  0.781000  0.654167  0.808333  0.724833  0.992833   
3  0.500000  0.781000  0.654167  0.808333  0.724833  0.992833  0.668833   
4  0.781000  0.654167  0.808333  0.724833  0.992833  0.668833  0.750000   

         7         8         9     ...           40        41        42  \
0  0.724833  0.992833  0.668833    ...     0.658333  0.500000  1.085000   
1  0.992833  0.668833  0.750000    ...     0.500000  1.085000  0.953333   
2  0.668833  0.750000  0.830833    ...     1.085000  0.953333  0.500000   
3  0.750000  0.830833  0.834500    ...     0.953333  0.500000  0.846000   
4  0.830833  0.834500  0.816667    ...     0.500000  0.846000  0.784667   

         43        44        45        46        47        48        49  
0  0.953333  0.500000  0.846000  0.784667  0.751667  0.247333  1.001667  
1  0.500000  0.846000  0.784667  0.751667  0.247333  1.001667  1.008333  
2  0.846000  0.784667  0.751667  0.247333  1.001667  1.008333  0.790500  
3  0.784667  0.751667  0.247333  1.001667  1.008333  0.790500  1.124667  
4  0.751667  0.247333  1.001667  1.008333  0.790500  1.124667  0.492167  

[5 rows x 50 columns]

In [11]:
msk = np.random.rand(len(df2)) < 0.8
train = df2[msk]
test = df2[~msk]
train
# then normalize
# then write to train.csv, test.csv df2.to_csv(...)
train.head()

0         1         2         3         4         5         6   \
0  0.948333  0.942500  0.912333  0.500000  0.781000  0.654167  0.808333   
1  0.942500  0.912333  0.500000  0.781000  0.654167  0.808333  0.724833   
2  0.912333  0.500000  0.781000  0.654167  0.808333  0.724833  0.992833   
3  0.500000  0.781000  0.654167  0.808333  0.724833  0.992833  0.668833   
4  0.781000  0.654167  0.808333  0.724833  0.992833  0.668833  0.750000   

         7         8         9     ...           40        41        42  \
0  0.724833  0.992833  0.668833    ...     0.658333  0.500000  1.085000   
1  0.992833  0.668833  0.750000    ...     0.500000  1.085000  0.953333   
2  0.668833  0.750000  0.830833    ...     1.085000  0.953333  0.500000   
3  0.750000  0.830833  0.834500    ...     0.953333  0.500000  0.846000   
4  0.830833  0.834500  0.816667    ...     0.500000  0.846000  0.784667   

         43        44        45        46        47        48        49  
0  0.953333  0.500000  0.846000  0.784667  0.751667  0.247333  1.001667  
1  0.500000  0.846000  0.784667  0.751667  0.247333  1.001667  1.008333  
2  0.846000  0.784667  0.751667  0.247333  1.001667  1.008333  0.790500  
3  0.784667  0.751667  0.247333  1.001667  1.008333  0.790500  1.124667  
4  0.751667  0.247333  1.001667  1.008333  0.790500  1.124667  0.492167  

[5 rows x 50 columns]

In [12]:
df2 = pd.read_csv('data/CNN_W2P_model/valid-1.csv', header = None) #, names = CSV_COLUMNS)
df2.describe()

0           1           2           3           4           5   \
count  100.000000  100.000000  100.000000  100.000000  100.000000  100.000000   
mean     0.834478    0.834212    0.828520    0.835327    0.833325    0.834935   
std      0.226580    0.226617    0.238085    0.229764    0.228138    0.230125   
min      0.208500    0.208500    0.083333    0.083333    0.083333    0.083333   
25%      0.707625    0.707625    0.707625    0.711417    0.711417    0.711417   
50%      0.837750    0.837750    0.837750    0.842333    0.842333    0.842333   
75%      0.971917    0.971917    0.971917    0.971917    0.968417    0.968417   
max      1.352000    1.352000    1.352000    1.352000    1.352000    1.352000   

               6           7           8           9      ...              40  \
count  100.000000  100.000000  100.000000  100.000000     ...      100.000000   
mean     0.829142    0.836500    0.840150    0.841215     ...        0.861833   
std      0.229196    0.228631    0.227486    0.227594     ...        0.192697   
min      0.083333    0.083333    0.083333    0.083333     ...        0.083333   
25%      0.707625    0.711417    0.711833    0.711833     ...        0.734708   
50%      0.837750    0.842333    0.843333    0.846667     ...        0.867500   
75%      0.968333    0.968417    0.968417    0.968417     ...        0.971792   
max      1.352000    1.352000    1.352000    1.352000     ...        1.315000   

               41          42          43          44          45          46  \
count  100.000000  100.000000  100.000000  100.000000  100.000000  100.000000   
mean     0.866928    0.866152    0.867867    0.865625    0.862798    0.859623   
std      0.191246    0.191221    0.190591    0.193574    0.193434    0.192047   
min      0.083333    0.083333    0.083333    0.083333    0.083333    0.083333   
25%      0.737333    0.737333    0.750500    0.750500    0.737333    0.737333   
50%      0.868333    0.867500    0.868333    0.868333    0.867500    0.862333   
75%      0.986167    0.986167    0.986167    0.986167    0.971792    0.968375   
max      1.315000    1.315000    1.315000    1.315000    1.315000    1.315000   

               47          48          49  
count  100.000000  100.000000  100.000000  
mean     0.857418    0.864093    0.858880  
std      0.189738    0.187701    0.186599  
min      0.083333    0.083333    0.083333  
25%      0.737333    0.750500    0.737333  
50%      0.862333    0.867500    0.862333  
75%      0.968333    0.968375    0.968333  
max      1.315000    1.315000    1.315000  

[8 rows x 50 columns]

In [13]:
%%bash
rm -rf /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer
mkdir /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer
touch /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/__init__.py

In [18]:
# read data and convert to needed format
def read_dataset(filename, mode, batch_size=512):
    def _input_fn():
        def decode_csv(row):
            # row is a string tensor containing the contents of one row
            features = tf.decode_csv(row, record_defaults=DEFAULTS)  # string tensor -> list of 50 rank 0 float tensors
            label = features.pop()  # remove last feature and use as label
            features = tf.stack(features)  # list of rank 0 tensors -> single rank 1 tensor
            return {TIMESERIES_COL: features}, label

        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        dataset = tf.data.Dataset.list_files(filename)
        # Read in data from files
        dataset = dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        dataset = dataset.map(decode_csv)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None  # loop indefinitely
            dataset = dataset.shuffle(buffer_size=10 * batch_size)
        else:
            num_epochs = 1  # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)
        return dataset.make_one_shot_iterator().get_next()

    return _input_fn

import tensorflow as tf
tf.reset_default_graph()

features = tf.placeholder(tf.float32, shape=[512, N_INPUTS])

X = tf.reshape(features,
               [-1, N_INPUTS, 1])  # as a 1D "sequence" with only one time-series observation (height)
c1 = tf.layers.conv1d(X, filters=N_INPUTS // 2,
                      kernel_size=3, strides=1,
                      padding='same', activation=tf.nn.relu)
p1 = tf.layers.max_pooling1d(c1, pool_size=2, strides=2)

c2 = tf.layers.conv1d(p1, filters=N_INPUTS // 2,
                      kernel_size=3, strides=1,
                      padding='same', activation=tf.nn.relu)
p2 = tf.layers.max_pooling1d(c2, pool_size=2, strides=2)

outlen = p2.shape[1] * p2.shape[2]
c2flat = tf.reshape(p2, [-1, outlen])
h1 = tf.layers.dense(c2flat, 3, activation=tf.nn.relu)
predictions = tf.layers.dense(h1, 1, activation=None)  # linear output: regression

N_OUTPUTS = 1
TIMESERIES_COL = 'price'
SEQ_LEN = 50

def init(hparams):
    global SEQ_LEN, DEFAULTS, N_INPUTS
    SEQ_LEN = hparams['sequence_length']
    DEFAULTS = [[0.0] for x in range(0, SEQ_LEN)]
    N_INPUTS = SEQ_LEN - N_OUTPUTS
    
init({'sequence_length': 50})
init = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init)
  features1, labels = sess.run(read_dataset('data/CNN_W2P_model/train-1.csv', None)())
#   features, labels = read_dataset('data/CNN_W2P_model/train-1.csv', None)()
  preds = sess.run(predictions, feed_dict={features: features1['price']})
# (preds*60)

## Testing LSTM shapes.

x = tf.reshape(features, [-1, N_INPUTS, 1])

# 2. configure the RNN
cell1 = tf.nn.rnn_cell.GRUCell(N_INPUTS * 2)
cell2 = tf.nn.rnn_cell.GRUCell(N_INPUTS // 2)
cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
# 'state' is now a tuple containing the final state of each cell layer
# we use state[1] below to extract the final state of the final layer
outputs = outputs[:, (N_INPUTS-1):, :] # last one only

# 3. flatten lstm output and pass through a dense layer
lstm_flat = tf.reshape(outputs, [-1, cells.output_size])
h1 = tf.layers.dense(lstm_flat, cells.output_size//2, activation=tf.nn.relu)
predictions_new = tf.layers.dense(h1, 1, activation=None) # (?, 1)

# 3. pass rnn output through a dense layer
h1 = tf.layers.dense(state[1], cells.output_size // 2, activation=tf.nn.relu)
predictions = tf.layers.dense(h1, 1, activation=None)  # (?, 1)

NameError: name 'N_INPUTS' is not defined

In [19]:
%%writefile /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/model.py
#!/usr/bin/env python

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

TIMESERIES_COL = 'price'
N_OUTPUTS = 1  # in each sequence, 1-49 are features, and 50 is label
SEQ_LEN = None
DEFAULTS = None
N_INPUTS = None


def init(hparams):
    global SEQ_LEN, DEFAULTS, N_INPUTS
    SEQ_LEN = hparams['sequence_length']
    DEFAULTS = [[0.0] for x in range(0, SEQ_LEN)]
    N_INPUTS = SEQ_LEN - N_OUTPUTS


def linear_model(features, mode, params):
    X = features[TIMESERIES_COL]
    predictions = tf.layers.dense(X, 1, activation=None)
    return predictions


def dnn_model(features, mode, params):
    X = features[TIMESERIES_COL]
    h1 = tf.layers.dense(X, 10, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, 3, activation=tf.nn.relu)
    predictions = tf.layers.dense(h2, 1, activation=None)  # linear output: regression
    return predictions


def cnn_model(features, mode, params):
    X = tf.reshape(features[TIMESERIES_COL],
                   [-1, N_INPUTS, 1])  # as a 1D "sequence" with only one time-series observation (height)
    c1 = tf.layers.conv1d(X, filters=N_INPUTS // 2,
                          kernel_size=3, strides=1,
                          padding='same', activation=tf.nn.relu)
    p1 = tf.layers.max_pooling1d(c1, pool_size=2, strides=2)

#     outlen = p1.shape[1] * p1.shape[2]
#     c1flat = tf.reshape(p1, [-1, outlen])
#     h1 = tf.layers.dense(c1flat, 3, activation=tf.nn.relu)
#     predictions = tf.layers.dense(h1, 1, activation=None)  # linear output: regression
#     return predictions
    
    c2 = tf.layers.conv1d(p1, filters=N_INPUTS // 2,
                          kernel_size=3, strides=1,
                          padding='same', activation=tf.nn.relu)
    p2 = tf.layers.max_pooling1d(c2, pool_size=2, strides=2)

    outlen = p2.shape[1] * p2.shape[2]
    c2flat = tf.reshape(p2, [-1, outlen])
    h1 = tf.layers.dense(c2flat, 3, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None)  # linear output: regression
    return predictions


def rnn_model(features, mode, params):
    CELL_SIZE = N_INPUTS // 3  # size of the internal state in each of the cells

    # 1. dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell = tf.nn.rnn_cell.GRUCell(CELL_SIZE)
    outputs, state = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)

    # 3. pass rnn output through a dense layer
    h1 = tf.layers.dense(state, N_INPUTS // 2, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, N_INPUTS // 2, activation=tf.nn.relu6)
    h3 = tf.layers.dense(h2, N_INPUTS // 2, activation=tf.nn.relu)
    
    predictions = tf.layers.dense(h3, 1, activation=None)  # (?, 1)
    return predictions


# 2-layer RNN
def rnn2_model(features, mode, params):
    # dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell1 = tf.nn.rnn_cell.GRUCell(N_INPUTS * 2)
    cell2 = tf.nn.rnn_cell.GRUCell(N_INPUTS // 2)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
    outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
    # 'state' is now a tuple containing the final state of each cell layer
    # we use state[1] below to extract the final state of the final layer
    outputs = outputs[:, (N_INPUTS-1):, :] # last one only

    # 3. flatten lstm output and pass through a dense layer
    lstm_flat = tf.reshape(outputs, [-1, cells.output_size])
    h1 = tf.layers.dense(lstm_flat, cells.output_size//2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None) # (?, 1)
    return predictions
  
    # 3. pass rnn output through a dense layer
    h1 = tf.layers.dense(state[1], cells.output_size // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None)  # (?, 1)
    return predictions


# create N-1 predictions
def rnnN_model(features, mode, params):
    # dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell1 = tf.nn.rnn_cell.GRUCell(N_INPUTS * 2)
    cell2 = tf.nn.rnn_cell.GRUCell(N_INPUTS // 2)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
    outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
    # 'outputs' contains the state of the final layer for every time step
    # not just the last time step (?,N_INPUTS, final cell size)
    
    # 3. pass state for each time step through a DNN, to get a prediction
    # for each time step 
    h1 = tf.layers.dense(outputs, cells.output_size, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, cells.output_size // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h2, 1, activation=None)  # (?, N_INPUTS, 1)
    predictions = tf.reshape(predictions, [-1, N_INPUTS])
    return predictions # return prediction for each time step


# read data and convert to needed format
def read_dataset(filename, mode, batch_size=512):
    def _input_fn():
        def decode_csv(row):
            # row is a string tensor containing the contents of one row
            features = tf.decode_csv(row, record_defaults=DEFAULTS)  # string tensor -> list of 50 rank 0 float tensors
            label = features.pop()  # remove last feature and use as label
            features = tf.stack(features)  # list of rank 0 tensors -> single rank 1 tensor
            return {TIMESERIES_COL: features}, label

        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        dataset = tf.data.Dataset.list_files(filename)
        # Read in data from files
        dataset = dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        dataset = dataset.map(decode_csv)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None  # loop indefinitely
            dataset = dataset.shuffle(buffer_size=10 * batch_size)
        else:
            num_epochs = 1  # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)
        return dataset.make_one_shot_iterator().get_next()

    return _input_fn


def serving_input_fn():
    feature_placeholders = {
        TIMESERIES_COL: tf.placeholder(tf.float32, [None, N_INPUTS])
    }

    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    features[TIMESERIES_COL] = tf.squeeze(features[TIMESERIES_COL], axis=[2])

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)


def compute_errors(features, labels, predictions):
    labels = tf.expand_dims(labels, -1)  # rank 1 -> rank 2 to match rank of predictions

    if predictions.shape[1] == 1:
        loss = tf.losses.mean_squared_error(labels, predictions)
        rmse = tf.metrics.root_mean_squared_error(labels, predictions)
        return loss, rmse
    else:
        # one prediction for every input in sequence
        # get 1-N of (x + label)
        labelsN = tf.concat([features[TIMESERIES_COL], labels], axis=1)
        labelsN = labelsN[:, 1:]
        # loss is computed from the last 1/3 of the series
        N = (2 * N_INPUTS) // 3
        loss = tf.losses.mean_squared_error(labelsN[:, N:], predictions[:, N:])
        # rmse is computed from last prediction and last label
        lastPred = predictions[:, -1]
        rmse = tf.metrics.root_mean_squared_error(labels, lastPred)
        return loss, rmse

# RMSE when predicting same as last value
def same_as_last_benchmark(features, labels):
    predictions = features[TIMESERIES_COL][:,-1] # last value in input sequence
    return tf.metrics.root_mean_squared_error(labels, predictions)


# create the inference model
def sequence_regressor(features, labels, mode, params):
    # 1. run the appropriate model
    model_functions = {
        'linear': linear_model,
        'dnn': dnn_model,
        'cnn': cnn_model,
        'rnn': rnn_model,
        'rnn2': rnn2_model,
        'rnnN': rnnN_model}
    model_function = model_functions[params['model']]
    print(model_function)
    predictions = model_function(features, mode, params)

    # 2. loss function, training/eval ops
    loss = None
    train_op = None
    eval_metric_ops = None
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        loss, rmse = compute_errors(features, labels, predictions)
    
        if mode == tf.estimator.ModeKeys.TRAIN:
            # this is needed for batch normalization, but has no effect otherwise
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                # 2b. set up training operation
                train_op = tf.contrib.layers.optimize_loss(
                    loss,
                    tf.train.get_global_step(),
                    learning_rate=params['learning_rate'],
                    optimizer="Adam")

        # 2c. eval metric
        eval_metric_ops = {
            "RMSE": rmse,
            "RMSE_same_as_last": same_as_last_benchmark(features, labels),
        }

    # 3. Create predictions
    if predictions.shape[1] != 1:
        predictions = predictions[:, -1]  # last predicted value
    predictions_dict = {"predicted": predictions}

    # 4. return EstimatorSpec
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions_dict,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops,
        export_outputs={
            'predictions': tf.estimator.export.PredictOutput(predictions_dict)}
    )


def train_and_evaluate(output_dir, hparams):
    get_train = read_dataset(hparams['train_data_path'],
                             tf.estimator.ModeKeys.TRAIN,
                             hparams['train_batch_size'])
    get_valid = read_dataset(hparams['eval_data_path'],
                             tf.estimator.ModeKeys.EVAL,
                             1000)
    estimator = tf.estimator.Estimator(model_fn=sequence_regressor,
                                       params=hparams,
                                       config=tf.estimator.RunConfig(
                                           save_checkpoints_steps=50,
                                         save_summary_steps=50
                                       ),
                                       model_dir=output_dir)
    train_spec = tf.estimator.TrainSpec(input_fn=get_train,
                                        max_steps=hparams['train_steps'])
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn=get_valid,
                                      steps=None,
                                      exporters=exporter,
                                      #start_delay_secs=hparams['eval_delay_secs'],
                                      #throttle_secs=hparams['min_eval_frequency']
                                     )
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

Overwriting /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/model.py


In [20]:
%%writefile /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/task.py

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Example implementation of code to run on the Cloud ML service.
"""

import traceback
import argparse
import json
import os
from . import model

if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  # Input Arguments
  parser.add_argument(
      '--train_data_path',
      help='GCS or local path to training data',
      required=True
  )
  parser.add_argument(
      '--eval_data_path',
      help='GCS or local path to evaluation data',
      required=True
  )
  parser.add_argument(
      '--train_batch_size',
      help='Batch size for training steps',
      type=int,
      default=100
  )
  parser.add_argument(
      '--learning_rate',
      help='Initial learning rate for training',
      type=float,
      default=0.01
  )
  parser.add_argument(
      '--train_steps',
      help="""\
      Steps to run the training job for. A step is one batch-size,\
      """,
      type=int,
      default=0
  )
  parser.add_argument(
      '--sequence_length',
      help="""\
      This model works with fixed length sequences. 1-(N-1) are inputs, last is output
      """,
      type=int,
      default=10
  )
  parser.add_argument(
      '--output_dir',
      help='GCS location to write checkpoints and export models',
      required=True
  )
  model_names = [name.replace('_model','') \
                   for name in dir(model) \
                     if name.endswith('_model')]
  parser.add_argument(
      '--model',
      help='Type of model. Supported types are {}'.format(model_names),
      required=True
  )
  parser.add_argument(
      '--job-dir',
      help='this model ignores this field, but it is required by gcloud',
      default='junk'
  )
  parser.add_argument(
      '--eval_delay_secs',
      help='How long to wait before running first evaluation',
      default=10,
      type=int
  )
  parser.add_argument(
      '--min_eval_frequency',
      help='Minimum number of training steps between evaluations',
      default=60,
      type=int
  )

  args = parser.parse_args()
  hparams = args.__dict__
  
  # unused args provided by service
  hparams.pop('job_dir', None)
  hparams.pop('job-dir', None)

  output_dir = hparams.pop('output_dir')

  # Append trial_id to path if we are doing hptuning
  # This code can be removed if you are not using hyperparameter tuning
  output_dir = os.path.join(
      output_dir,
      json.loads(
          os.environ.get('TF_CONFIG', '{}')
      ).get('task', {}).get('trial', '')
  )

  # calculate train_steps if not provided
  if hparams['train_steps'] < 1:
     # 1,000 steps at batch_size of 100
     hparams['train_steps'] = (1000 * 100) // hparams['train_batch_size']
     print ("Training for {} steps".format(hparams['train_steps']))

  model.init(hparams)

  # Run the training job
  model.train_and_evaluate(output_dir, hparams)


Overwriting /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/task.py


In [21]:
! pwd
#! ls
#!{PWD}/CNN_W2P_model/trainer

! ls /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer

/content/datalab/energy_forcasing/Sarah
__init__.py  model.py  task.py


## run the model locally on the vm

In [22]:
%%bash

DATADIR=$(pwd)/data/CNN_W2P_model 
OUTDIR=$(pwd)/CNN_W2P_model/trained
SEQ_LEN=50
rm -rf $OUTDIR
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/CNN_W2P_model/trainer \
   -- \
   --train_data_path="${DATADIR}/train-1.csv" \
   --eval_data_path="${DATADIR}/valid-1.csv"  \
   --output_dir=${OUTDIR} \
   --model=dnn --train_steps=2000  --sequence_length=$SEQ_LEN

<function dnn_model at 0x7fe506f37320>
<function dnn_model at 0x7fe506f37320>
<function dnn_model at 0x7fe506f37320>


/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--train_data_path=/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/train-1.csv', u'--eval_data_path=/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/valid-1.csv', u'--output_dir=/content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trained', u'--model=dnn', u'--train_steps=2000', u'--sequence_length=50'], u'job_name': u'trainer.task'}, u'task': {}}
INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec':

In [23]:
ls CNN_W2P_model/trained/export/exporter/

1537556029/


In [26]:
%%writefile test.json
[45.52, 45.24, 43.792, 24.0, 37.488, 31.400000000000002, 38.800000000000004, 34.792, 47.656000000000006, 32.104000000000006, 36.0, 39.88, 40.056000000000004, 39.2, 34.952, 50.42400000000001, 36.472, 42.848000000000006, 30.496, 44.712, 32.128, 52.08, 31.664, 47.88, 44.0, 51.352000000000004, 42.624, 54.879999999999995, 32.800000000000004, 31.400000000000002, 30.680000000000003, 44.608000000000004, 32.256, 50.072, 37.424, 39.88, 47.752, 44.800000000000004, 52.08, 46.952, 31.6, 24.0, 52.08, 45.760000000000005, 24.0, 40.608000000000004, 37.664, 36.080000000000005, 11.872]

Overwriting test.json


In [27]:
%bash
model=$(ls CNN_W2P_model/trained/export/exporter | tail -1)
gcloud ml-engine local predict \
    --model-dir=CNN_W2P_model/trained/export/exporter/$model \
    --json-instances=./test.json

PREDICTED
[6.672857761383057]


  from ._conv import register_converters as _register_converters
2018-09-21 18:54:24.660602: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA



In [28]:
%%writefile test.json
{"price": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}

Overwriting test.json


In [29]:
%bash
model=$(ls CNN_W2P_model/trained/export/exporter | tail -1)
gcloud ml-engine local predict \
    --model-dir=CNN_W2P_model/trained/export/exporter/$model \
    --json-instances=./test.json

PREDICTED
[0.5992636680603027]


  from ._conv import register_converters as _register_converters
2018-09-21 18:54:27.382280: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA



In [31]:
from google.datalab.ml import TensorBoard
TensorBoard().start('CNN_W2P_model'.format(BUCKET))

TensorBoard was started successfully with pid 28794. Click here to access it.

28794

# Make training and test data for the cloud

In [ ]:
import shutil
shutil.rmtree('/data/CNN_W2P_model ', ignore_errors=True)
#os.makedirs('/data/CNN_W2P_model/')

#for i in xrange(0,3):
#  to_csv('data/CNN_W2P_model/train-{}.csv'.format(i), N = 1000, J = 50)  # 1000 sequences
#  to_csv('data/CNN_W2P_model/valid-{}.csv'.format(i), N = 100, J = 50)

In [ ]:
%bash
gsutil -m rm -rf gs://${BUCKET}/CNN_W2P_model/*
gsutil -m cp data/CNN_W2P_model/*.csv gs://${BUCKET}/CNN_W2P_model

### Push out to the cloud

In [34]:
%%bash
for MODEL in dnn ; do ## rnn rnn2 rnnN; do ## linear dnn 
  OUTDIR=gs://${BUCKET}/CNN_W2P_model/${MODEL}
  JOBNAME=CNN_W2P_model_${MODEL}$(date -u +%y%m%d_%H%M%S)
  SEQ_LEN=50
  REGION=us-central1
  gsutil -m rm -rf $OUTDIR
  echo $JOBNAME
  gcloud ml-engine jobs submit training $JOBNAME \
     --region=$REGION \
     --module-name=trainer.task \
     --package-path=${PWD}/CNN_W2P_model/trainer \
     --job-dir=$OUTDIR \
     --scale-tier=PREMIUM_1 \
     --runtime-version=$TFVERSION \
     -- \
     --train_data_path="gs://${BUCKET}/CNN_W2P_model/train-1.csv" \
     --eval_data_path="gs://${BUCKET}/CNN_W2P_model/valid-1.csv"  \
     --output_dir=$OUTDIR \
     --train_steps=500 --sequence_length=$SEQ_LEN --model=$MODEL
done


CNN_W2P_model_dnn180921_185544
jobId: CNN_W2P_model_dnn180921_185544
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [CNN_W2P_model_dnn180921_185544] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe CNN_W2P_model_dnn180921_185544

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs CNN_W2P_model_dnn180921_185544


In [37]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/CNN_W2P_model'.format(BUCKET))

TensorBoard was started successfully with pid 31033. Click here to access it.

31033

In [35]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

Stopped TensorBoard with pid 28744
Stopped TensorBoard with pid 28794
Stopped TensorBoard with pid 29716


In [ ]:
## command line access of the TB
print('tensorboard --logdir=gs://{}/CNN_W2P_model --port=8083'.format(BUCKET))

# Build a yaml

In [ ]:
!pwd

In [ ]:
%%writefile  hyperparam.yaml
trainingInput:
  hyperparameters:
    goal: MINIMIZE
    maxTrials: 5
    maxParallelTrials: 1
    hyperparameterMetricTag: rmse
    params:
    - parameterName: learning_rate
      type: DOUBLE
      minValue: 0.01
      maxValue: 0.1
      scaleType: UNIT_LOG_SCALE

In [ ]:
%%bash

MODEL=dnn
OUTDIR=gs://${BUCKET}/CNN_W2P_model/${MODEL}
JOBNAME=CNN_W2P_model_${MODEL}$(date -u +%y%m%d_%H%M%S)

SEQ_LEN=50
REGION=us-central1

#gsutil -m rm -rf $OUTDIR
echo $JOBNAME

gcloud ml-engine jobs submit training $JOBNAME \
     --region=$REGION \
     --module-name=trainer.task \
     --package-path=${PWD}/CNN_W2P_model/trainer \
     --job-dir=$OUTDIR \
     --scale-tier=PREMIUM_1 \
     --runtime-version=$TFVERSION \
     --config=hyperparam.yaml \
     -- \
     --train_data_path="gs://${BUCKET}/CNN_W2P_model/train-1.csv" \
     --eval_data_path="gs://${BUCKET}/CNN_W2P_model/valid-1.csv"  \
     --output_dir=$OUTDIR \
     --train_steps=5000 --sequence_length=$SEQ_LEN --model=$MODEL
done

In [ ]:
!gcloud ml-engine jobs describe CNN_W2P_model_dnn180921_165257

## Deploy the model

### Convert the data into something that can be read into the model, so strings that are 49 long where the 50th price is perdicted.

In [65]:
%bash
gsutil ls gs://${BUCKET}/CNN_W2P_model/dnn/export/exporter/

gs://sarah-bucket/CNN_W2P_model/dnn/export/exporter/
gs://sarah-bucket/CNN_W2P_model/dnn/export/exporter/1537556449/


In [85]:
%bash
MODEL_NAME="CNN_W2P_model"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/CNN_W2P_model/dnn/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

Deleting and deploying CNN_W2P_model ml_on_gcp from gs://sarah-bucket/CNN_W2P_model/dnn/export/exporter/1537556449/ ... this will take a few minutes


Created ml engine model [projects/qwiklabs-gcp-aebfb78fe0f1b1d1/models/CNN_W2P_model].
Creating version (this might take a few minutes)......
............................................................................................done.


Make data to feed to model

In [88]:
def to_model_csv(filename, N, J):
  with open(filename, 'w') as ofp:
    N = df5.shape[0] - J
    for ii in xrange(0, N):
      seq = df5.price[(0+ii):(J+ii)]
      line = ",".join(map(str, seq))
      ofp.write(line + '\n')
      #print(len(seq))
      if len(seq) != J:
        break

import os
try:
  os.makedirs('data/CNN_W2P_model/')
except OSError:
  pass

to_model_csv('data/CNN_W2P_model/to_model.csv', N = 1000, J = 49) 


In [89]:
model_data = pd.read_csv('/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/to_model.csv', header = None)
model_data.shape

(6597, 49)

In [92]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME="CNN_W2P_model_dnn"
MODEL_VERSION="ml_on_gcp"

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
    {
      'price': prices
    },
  ]
}

In [93]:
## 55.11,62.0,46.79,44.0,44.0,44.86,60.89,58.69,36.49,57.69,51.2,48.79,48.35,42.2,46.37,62.5,62.5,54.0,52.54,62.18,48.5,55.35,52.0,40.97,42.6,64.1,61.59,48.9,37.4,45.93,37.3,65.48,62.5,65.03,57.07,23.35,54.56,28.02,39.84,51.0,42.46,47.02,37.1,38.0,38.0,60.99,50.01,19.52,39.98,47.96

This will take a few minutes.

In [94]:
#result = []
for y in range(1,10):#model_data.shape[0]):
    raw_prices = model_data.iloc[y]
    prices = [float(price) for price in raw_prices]
    #print prices
    data = {
      'instances': [
        {
          'price': prices
        },
      ]
    }
    #print(data)
    response = requests.post(api, json=data, headers=headers)
    #print y/model_data.shape[0]
    forecast = response.json()##['predictions'][0]['predicted'][0] ## (df5.price[(nn):(nn+J)])/60
    print (forecast)
    #result.append(response.content[32:-4])

{u'error': {u'status': u'NOT_FOUND', u'message': u'Field: name Error: The model resource: "CNN_W2P_model_dnn" was not found. Please create the Cloud ML model resource first by using \'gcloud ml-engine models create CNN_W2P_model_dnn\'.', u'code': 404, u'details': [{u'fieldViolations': [{u'field': u'name', u'description': u'The model resource: "CNN_W2P_model_dnn" was not found. Please create the Cloud ML model resource first by using \'gcloud ml-engine models create CNN_W2P_model_dnn\'.'}], u'@type': u'type.googleapis.com/google.rpc.BadRequest'}]}}
{u'error': {u'status': u'NOT_FOUND', u'message': u'Field: name Error: The model resource: "CNN_W2P_model_dnn" was not found. Please create the Cloud ML model resource first by using \'gcloud ml-engine models create CNN_W2P_model_dnn\'.', u'code': 404, u'details': [{u'fieldViolations': [{u'field': u'name', u'description': u'The model resource: "CNN_W2P_model_dnn" was not found. Please create the Cloud ML model resource first by using \'gcloud 

In [ ]:
print result

In [ ]:
!pwd
!ls

In [ ]:
%%writefile  hyperparam.yaml
trainingInput:
  hyperparameters:
    goal: MINIMIZE
    maxTrials: 5
    maxParallelTrials: 1
    hyperparameterMetricTag: rmse
    params:
    - parameterName: learning_rate
      type: DOUBLE
      minValue: 0.01
      maxValue: 0.1
      scaleType: UNIT_LOG_SCALE